In [ ]:
import calcbench as cb
import pandas as pd
import datetime
pit_columns = ['CIK', 'calendar_period', 'calendar_year', 'date_reported', 
               'fiscal_period', 'fiscal_year', 'metric', 'revision_number','ticker', 'value']

In [ ]:
tickers = cb.companies(index='SP500').ticker

In [ ]:
metrics = cb.available_metrics()
face_metrics = [m['metric'] for m in metrics['face']]

In [ ]:
ticker_groups = [list(tickers[i:i + 100]) for i in range(0, len(tickers), 100)]

In [ ]:
face_metrics = ['Assets']

In [ ]:
data = pd.DataFrame()
elapsed_start = datetime.datetime.now()
for company_number, ticker_group in enumerate(ticker_groups):
    start = datetime.datetime.now()
    try:
        pit_data = cb.point_in_time(metrics=face_metrics, company_identifiers=list(ticker_group), all_history=True)
    except Exception as e:
        print("Exception getting {0}".format(ticker_group))
        raise e
    else:
        if not pit_data.empty:
            data = data.append(pit_data[pit_columns])
        continue
        print("done getting {0}, it took {1}, elapsed {2}, company_number {3}, average time {4}"\
              .format(ticker_group, 
                      datetime.datetime.now() - start,
                      datetime.datetime.now() - elapsed_start,
                      company_number,
                        (datetime.datetime.now() - elapsed_start) / (company_number + 1)))

In [ ]:
missing_periods = []
tickers = data.ticker.unique()
#tickers = ['ALLE']
for ticker in tickers:
    test_data = data[(data.ticker == ticker) & (data.revision_number == 0)][['calendar_year', 'calendar_period', 'ticker']]
    test_dicts = [test_data.loc[i].to_dict() for i in test_data.index]
    last_row = test_dicts[0]
    for row in test_dicts[1:]:
        if last_row['calendar_period'] == 0:
            if row['calendar_period'] != 1:
                missing_periods.append(last_row)
        elif last_row['calendar_period'] == 4:
            if row['calendar_period'] != 0:
                missing_periods.append(last_row)
        elif last_row['calendar_period'] + 1 != row['calendar_period']:
            missing_periods.append(last_row)
        last_row = row

In [ ]:
sorted(missing_periods, key=lambda row : row['calendar_year'], reverse=True)

In [ ]:
data.calendar_period.value_counts()

In [ ]:
dir(data.calendar_period)